In [ ]:
import networkx as nx
import cvxopt as cvx
import cvxopt.lapack
import numpy as np
import picos as pic
import networkx as nx
import pylab
import random
import time
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

import matplotlib
warnings.filterwarnings('ignore')
# 解の精度をより高める．（値を小さくするほど正確になる．）
cvxopt.solvers.options['abstol'] = 1e-12
cvxopt.solvers.options['reltol'] = 1e-10
cvxopt.solvers.options['feastol'] = 1e-10

# 計算状況（プログレスバー）を表示させないようにする．
cvxopt.solvers.options['show_progress'] = False
for N in range(100, 110, 10):
    M = N*2
    C = nx.gnm_random_graph(N, M)x

    """
    picosのやつに、add_edges_fromをし、d-waveのgraphのベースを.LCF_graphにする
    (.LCF_graph+add_edges_from）でグラフを統一
    """
    # Use a fixed RNG seed so the result is reproducable.
    random.seed(1)

    # Number of nodes.
    # # Add edges to the graph (also adds nodes)​
    G = nx.LCF_graph(N, [0, 0], int(N/2))
    for x in C.edges():
        G.add_edge(x[0], x[1])
    G = nx.DiGraph(G)  # edges are bidirected

    # Generate edge capacities.
    c = {}
    for e in sorted(G.edges(data=True)):
        capacity = 0.5
        e[2]['capacity'] = capacity
        c[(e[0], e[1])] = capacity

    # Convert the capacities to a PICOS expression.
    cc = pic.new_param('c', c)

    # 円周上に座標配置
    pos = {
        n: (np.cos(2*i*np.pi/N), np.sin(2*i*np.pi/N))
        for i, n in enumerate(G.nodes)
    }
    # randomに座標設定
    # pos = nx.random_layout(G)
    # Define a plotting helper that closes the old and opens a new figure.

    def new_figure():
        try:
            global fig
            pylab.close(fig)
        except NameError:
            pass
        fig = pylab.figure(figsize=(11, 8))
        fig.gca().axes.get_xaxis().set_ticks([])
        fig.gca().axes.get_yaxis().set_ticks([])

    # Make G undirected.
    G = nx.Graph(G)
    # Allocate weights to the edges.
    for (i, j) in G.edges():
        G[i][j]['weight'] = c[i, j]+c[j, i]
    maxcut = pic.Problem()
    # Add the symmetric matrix variable.
    X = maxcut.add_variable('X', (N, N), 'symmetric')
    # Retrieve the Laplacian of the graph.
    LL = 1/4.*nx.laplacian_matrix(G).todense()
    L = pic.new_param('L', LL)
    # Constrain X to have ones on the diagonal.
    maxcut.add_constraint(pic.diag_vect(X) == 1)
    # Constrain X to be positive semidefinite.
    maxcut.add_constraint(X >> 0)
    # Set the objective.
    maxcut.set_objective('max', L | X)
    # Solve the problem.
    start = time.time()
    maxcut.solve(solver='cvxopt')
    # print('bound from the SDP relaxation: {0}'.format(maxcut.obj_value()))
    # Use a fixed RNG seed so the result is reproducable.
#     cvx.setseed(1)
    # Perform a Cholesky factorization.
    V = X.value
    cvxopt.lapack.potrf(V)
    for i in range(N):
        for j in range(i+1, N):
            V[i, j] = 0
    # Do up to 100 projections. Stop if we are within a factor 0.878 of the SDP
    # optimal value.
    count = 0
    obj_sdp = maxcut.obj_value()
    obj = 0
    while (count < 1000 or obj < 0.87856*obj_sdp):
        r = cvx.normal(N, 1)
        x = cvx.matrix(np.sign(V*r))
        o = (x.T*L*x).value
        if o > obj:
            x_cut = x
            obj = o
        count += 1
    x = x_cut
    elapsed_time = time.time() - start
    cut = [(i, j) for (i, j) in G.edges() if x[i]*x[j] < 0]
    sval = sum(G[e[0]][e[1]]['weight'] for e in cut)
    print("classic(N: {} M: {}): {}".format(N, M, sval))
    print("classic:elapsed_time:{0}".format(elapsed_time) + "[sec]")
    # ------- Set up our graph -------
    start = time.time()
    # Create empty graph
    G = nx.LCF_graph(N, [0, 0], int(N/2))

    # Add edges to the graph (also adds nodes)
    for x in C.edges():
        G.add_edge(x[0], x[1])

    # ------- Set up our QUBO dictionary -------

    # Initialize our Q matrix
    Q = defaultdict(int)

    # Update Q matrix for every edge in the graph
    for i, j in G.edges:
        Q[(i, i)] += -1
        Q[(j, j)] += -1
        Q[(i, j)] += 2

    # ------- Run our QUBO on the QPU -------
    # Set up QPU parameters
    chainstrength = 1
    numruns = 1

    # Run the QUBO on the solver from your config file
    sampler = EmbeddingComposite(DWaveSampler(token="DEV-2d1f56ea19923664f36dd36fde103fe781b74d14"))
    start = time.time()
    response = sampler.sample_qubo(
        Q, chain_strength=chainstrength, num_reads=numruns)
    energies = iter(response.data())
    # ------- Print results to user -------
    for line in response:
        S0 = [k for k, v in line.items() if v == 0]
        S1 = [k for k, v in line.items() if v == 1]
        E = next(energies).energy
        print("Quantum(N: {} M: {}): {}".format(N, M, str(int(-1*E))))
        elapsed_time = time.time() - start
        print("Quantum:elapsed_time:{0}".format(elapsed_time) + "[sec]")
        break